#Lecture 6.2: Training a Factorization Machine
##What you'll learn in this session
1.  Reduce Rendel's equations to code
2.  Use what you've learned to improve performance
3.  How to adapt the technique for different types of prediction problems

##Order of topics
1.  Walk through code and compare to Rendle's paper
2.  Develop equations for least squares regression
3.  Adding L2 penalty to problem statement
4.  Checking code for correctness
5.  Initialization
6.  Modifying equations for different problem types.  

##Pre-reading
http://www.ismll.uni-hildesheim.de/pub/pdfs/Rendle2010FM.pdf - #Rendle's original paper

##Code Walk-through
The code in the code section below encapsulates Rendles equations 1 through 4.  A good place to start is the function "predict".  That shows python numpy code for generating predictions given the various matrix quantities in Rendles equation 1.  Equation 1 has four basic quantities - a scalar bias ($w_0$), a vector of linear weights ($w_1$ through $w_n$), factor matrix ($v$) and an attribute vectore ($x_1$ through $x_n$).  In the code the notations for the scalar bias is changed from $w_0$ to b.  Other than that the variable names in the code match those in the paper.  The code develops three scalar quantities whose sum constitutes the prediction - one quantity for each of the expressions in Rendle's equation 1.  The first is just the bias values b.  The second is denoted as $a_2$ and the third as $a_3$.  The quantity a2 is the linear term and is simply the dot product of the weight array w with the array of attributes x.  The quantity a3 is the quadratic term and involves the vectors that the paper denotes as $v_i$ in bold letters.  These vectors form the rows of a matrix denoted as bold V (upper case).  The sum involving the attribute vector is equivalent to a quadratic form involving the attribute vector x and a matrix $M = VV^T$.  The tricky bit is that the sum is only over the upper triangle of M.  It doesn't include the diagonal.  In the math for Lemma 3.1, Rendle shows that the third sum in equation 3, is equivalent to forming the quadratic form $x^TVV^Tx$, subtracting the diagonal contribution and then dividing by two.  You'll see that in the code.  First $VV^T$ is formed and then the diagonal elements are set to zero (instead of substracting them after the multiplication).  Then the quadratic form is evaluated and multiplied by a half.  

The function named "grad" implements equation 4 from the paper.  The gradients with respect to the bias term and the linear term are straightforward.  The implementation of the gradient with respect to the matrix V follows equation 4 except that it builds the matrix a row at a time whereas the equation in the paper is for single elements from V.  There is also a numerical version of the gradient function in order to check calculations.  The numerical gradient is a time consuming way to do the calculation, but the computation is a very straightforward implementation of the definition of a gradient so it's easy to check visually.

##In class exercises
1  The code for predict and grad define how predictions are made and the gradient of the predictions, but gradient descent needs the gradient of some performance measure with respect to the parameters (b, w and V in this case).  Suppose that the problem being solved is a regression problem and that the performance measure is mean squared error or a training set or test set.  Call the prediction function $P(b, w, V, x)$.  Write the expression for the performance over a set of examples and the gradient of the performance in terms of the gradient of $P$.  

2  How would your answer in the exercise above change if you chose to minimize mean absolute error instead of mean squared error?

3  The matrix V introduces a large number of free parameters into the solution.  It is likely that some sort of regularization will be required to avoid overfitting.  L2 is the easiest type of regularization to append to the problem.  To see why, write an expression for the L2 penalty of an array of weights and then take its gradient in symbolic form.  If you wanted to add an L2 penalty, what would you add to the gradient of the performance measure?  

4  Run the code below and adjust the parameters to improve the performance. 

In [2]:
__author__ = 'mike-bowles'
import urllib2
import numpy as np
from math import sqrt
import matplotlib.pyplot as plt

def predict(b, w, v, x): #b=scalar, w=np.array, v=np 2-dim array
    #dimensions
    #dim(x) = nAtt
    #dim(b) = python float
    #dim(w) = nAtt
    #dim(v) = nAtt x nDim
    #function return = nRow x 1
    a2 = np.dot(w, x)
    M = np.dot(v, np.transpose(v))
    i,j = np.indices(M.shape)
    M[i==j] = 0.0
    a3 = 0.5 * np.dot(x, np.dot(M, x))
    return b + a2 + a3

def grad(b, w, v, x):
    #x is single row from data in array form (not single column matrix
    #call of the form grad(,,,X[i,:])
    bGrad = 1.0
    wGrad = np.array(x)
    vGrad = np.zeros_like(v)
    vSum = np.zeros_like(v[1, :])
    for j in range(len(x)):
        vSum += v[j, :] * x[j]
    for i in range(len(x)):
        vGrad[i, :] = x[i] * vSum - v[i, :] * x[i] * x[i]
    return bGrad, wGrad, vGrad

def gradNum(b, w, v, x):
    eps = max(b * 1e-4, 1e-6)
    pr = predict(b, w, v, x)
    pbPlus = predict(b + eps, w, v, x)
    bGrad = (pbPlus - pr) / eps
    wGrad = np.zeros_like(w)
    for i in range(len(w)):
        eps = max(w[i] * 1e-4, 1e-6)
        wPlus = np.array(w)
        wPlus[i] = w[i] + eps
        pwPlus = predict(b, wPlus, v, x)
        wGrad[i] = (pwPlus - pr) / eps
    (vi, vj) = v.shape
    vGrad = np.zeros_like(v)
    for i in range(vi):
        for j in range(vj):
            eps = max(v[i,j] * 1e-4, 1e-6)
            vPlus = np.array(v)
            vPlus[i, j] = v[i,j] + eps
            pvPlus = predict(b, w, vPlus, x)
            vGrad[i,j] = (pvPlus - pr) / eps
    return bGrad, wGrad, vGrad

#read data
target_url = "http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv"
data = urllib2.urlopen(target_url)
x = []
y = []
names = []
firstLine = True
for row in data:
    if firstLine:
        names = row.strip().split(";")
        firstLine = False
    else:
        rowSplit = row.strip().split(";")
        y.append(float(rowSplit.pop()))

        floatRow = [float(num) for num in rowSplit]
        x.append(floatRow)

#training and test sets
indices = range(len(x))
xTe = [x[i] for i in indices if i%3 == 0 ]
xTr = [x[i] for i in indices if i%3 != 0 ]
yTe = [y[i] for i in indices if i%3 == 0]
yTr = [y[i] for i in indices if i%3 != 0]

xTe = np.array(xTe); xTr = np.array(xTr); yTe = np.array(yTe); yTr = np.array(yTr)

#meta parameters
nDim = 5   #dimension of abstract factor space
lam1 = 0.001 #multiplier for L2 penalty on linear weight vector w
lam2 = 0.1 #multiplier for L2 penalty on cross-product terms v
nPasses = 100 #number of passes through gradient descent calculations
gradTest = False  #test gradient calculation against numerical gradient?
gradStep = 0.0001  #step size for gradient descent.

#some setup and initialization
rowTr, nAtt = xTr.shape
rowTe, nAtt = xTe.shape
vSize = nDim * nAtt
wSize = nAtt
v = np.random.normal(0.0, 1.0 / float(vSize), (len(xTr[0, :]), nDim))
#v = np.zeros((len(xTr[0, :]), nDim))
b = 0.0
#w = np.random.normal(0.0, 1.0 / float(wSize), (len(xTr[0, :])))
w = np.zeros_like(xTr[0, :])

print 'Training Error     Test Error'
for iPass in range(nPasses):
    #calculate full batch gradient (whole training set)
    bGrad = 0.0; wGrad = np.zeros_like(xTr[0, :]); vGrad = np.zeros_like(v)
    for i in range(rowTr):
        x = xTr[i, :]
        bg, wg, vg = grad(b, w, v, x)
        if gradTest:
            bgNum, wgNum, vgNum = gradNum(b, w, v, x)
            print bg - bgNum, np.linalg.norm(wg - wgNum), np.linalg.norm(vg - vgNum)
        err = yTr[i] - predict(b, w, v, x)
        bGrad -= err * bg/float(rowTr)
        wGrad -= err * wg/float(rowTr)
        vGrad -= err * vg/float(rowTr)
    #include regularization penalties and take the step
    b -= gradStep * bGrad
    w -= gradStep * (wGrad + lam1 * w)
    v -= gradStep * (vGrad + lam2 * v)
    #calc and print result to monitor progress
    eTr = 0.0
    eTe = 0.0
    for i in range(rowTr):
        x = xTr[i, :]
        err = (yTr[i] - predict(b, w, v, x))
        eTr += err * err / rowTr
    for i in range(rowTe):
        x = xTe[i, :]
        err = (yTe[i] - predict(b, w, v, x))
        eTe += err * err / rowTe
    if iPass%10 == 0: print sqrt(eTr), sqrt(eTe)

print 'final train and test error = ', sqrt(eTr), sqrt(eTe)
print 'b = ', b
print 'w = ', w
print 'v = '
print v

Training Error     Test Error
3.67058181397 3.80384215942
2.76747788435 2.87455673336
2.27522773623 2.36769409713
5.32837041316 5.37935705912
2.21039301464 2.3163793714
1.95227755383 2.04459715013
1.72256840613 1.80796931797
1.51848414245 1.59782465876
1.34875246644 1.41818285787
1.22282712222 1.27821241807
final train and test error =  1.1451262263 1.18665923168
b =  0.0103010095673
w =  [ 0.08570646  0.00489227  0.00262736  0.02231197  0.00081567  0.08138491
  0.02534169  0.01026169  0.03439232  0.00678789  0.11211537]
v = 
[[  1.79792795e-02   3.57217874e-02   5.69684141e-02   4.54583888e-02
   -3.78942957e-02]
 [  5.75565269e-03   1.14820449e-02   6.19195338e-03  -1.76075195e-02
    2.56073621e-02]
 [  3.52021622e-02  -5.14779896e-03   9.50928997e-03  -3.91181685e-03
   -6.26953144e-03]
 [ -4.91828663e-03   2.03072014e-02  -2.19503209e-02   4.87997669e-02
   -2.48129519e-02]
 [ -5.23854890e-03   5.80503616e-03  -3.00243792e-02  -1.92368586e-02
   -1.03696782e-02]
 [  3.18649320e-02

##In-class coding
1.  Modify the code above to minimize mean absolute error instead of mean squared error. 


##Homework problems
1  Implement backtracking line search to improve gradient descent behavior

2  Suppose you're given a classification problem.  Suppose y takes values +/- 1.0 and call the prediction function f(x).  Produce the code that takes the gradients of the prediction function wrt to b, w and v (named bGrad, wGrad and vGrad in the code above) and calculates the loss function gradients for logistic loss ($\frac{1}{ln\,2}ln(1\,+\,e^{-y\,f(x)})$)

3  With the same suppositions as in 2, produce the code for hinge loss function ($max(0.0, 1\,-\,y\,f(x))$). 
